To perform limb detection, we will be using a modified MPII Human Pose dataset. First, we'll need to download the entire dataset. This file seeks to filter and shrink this dataset into a smaller subset for easier use. This does not update the dataset to have bounding boxes either.

In [ ]:
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz

--2021-03-27 19:24:32--  https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz
Resolving datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)... 139.19.206.177
Connecting to datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)|139.19.206.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12088943206 (11G) [application/x-gzip]
Saving to: ‘mpii_human_pose_v1.tar.gz.1’

mpii_human_pose_v1. 100%[===================>]  11.26G  18.2MB/s    in 10m 39s 

2021-03-27 19:35:11 (18.0 MB/s) - ‘mpii_human_pose_v1.tar.gz.1’ saved [12088943206/12088943206]



In [ ]:
!tar xzvf mpii_human_pose_v1.tar.gz

Streaming output truncated to the last 5000 lines.
./images/019279044.jpg
./images/068025069.jpg
./images/043439489.jpg
./images/065199085.jpg
./images/043180510.jpg
./images/077936327.jpg
./images/032166247.jpg
./images/085835606.jpg
./images/028663685.jpg
./images/085061606.jpg
./images/045571938.jpg
./images/052659494.jpg
./images/040132544.jpg
./images/041967229.jpg
./images/070550266.jpg
./images/006644163.jpg
./images/004278327.jpg
./images/061707653.jpg
./images/046332890.jpg
./images/038161650.jpg
./images/014551548.jpg
./images/012610270.jpg
./images/030097552.jpg
./images/090687283.jpg
./images/092028594.jpg
./images/020276921.jpg
./images/086201772.jpg
./images/081240248.jpg
./images/045623651.jpg
./images/002182340.jpg
./images/009557537.jpg
./images/039847876.jpg
./images/001289747.jpg
./images/062442493.jpg
./images/020601740.jpg
./images/023538326.jpg
./images/087342324.jpg
./images/061328639.jpg
./images/045247005.jpg
./images/043498418.jpg
./images/087985677.jpg
./imag

In [ ]:
import pandas as pd, csv
import os
import shutil

The MPII dataset annotations are stored in in a Matlab file structure, which makes it difficult to work with in Python and Jupyter Notebooks. Thank you to Github user Arifkhan21 (https://github.com/Arifkhan21/MPII-Dataset-in-CSV) for converting the annotations to a csv file.

In [118]:
dataset = pd.read_csv("https://raw.githubusercontent.com/Arifkhan21/MPII-Dataset-in-CSV/master/mpii_dataset.csv")

In [119]:
dataset

,Unnamed: 0,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,l ankle_X,l ankle_Y,pelvis_X,pelvis_Y,thorax_X,thorax_Y,upper neck_X,upper neck_Y,head top_X,head top_Y,r wrist_X,r wrist_Y,r elbow_X,r elbow_Y,r shoulder_X,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,0,015601864.jpg,620,394,616,269,573,185,647,188,661,221,656,231,610,187,647,176,637.0201,189.8183,695.9799,108.1817,606,217,553,161,601,167,692,185,693,240,688,313,3.021046,curling,sports
1,0,015599452.jpg,-1,-1,-1,-1,806,543,720,593,-1,-1,-1,-1,763,568,683,290,682.0000,256.0000,676.0000,68.0000,563,296,555,410,647,281,719,299,711,516,545,466,5.641276,curling,sports
2,0,005808361.jpg,804,711,816,510,908,438,1040,454,906,528,883,707,974,446,985,253,982.7591,235.9694,962.2409,80.0306,869,214,798,340,902,253,1067,253,1167,353,1142,478,4.718488,curling,sports
3,0,086617615.jpg,301,461,305,375,201,340,294,342,335,370,331,455,248,341,279,263,277.0210,268.7786,305.9790,184.2214,328,354,260,335,244,261,314,264,327,320,362,346,2.681349,curling,sports
4,0,060111501.jpg,980,322,896,318,865,248,943,226,948,290,881,349,904,237,858,135,871.1877,180.4244,835.8123,58.5756,772,294,754,247,792,147,923,123,995,163,961,223,3.806403,curling,sports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17367,0,033474347.jpg,876,461,815,380,779,296,742,303,706,386,739,468,761,300,684,219,671.7848,204.2324,629.2152,152.7676,693,243,726,260,704,214,663,223,674,286,631,260,2.003677,pushing car,transportation
17368,0,082650067.jpg,1038,656,950,504,903,328,831,337,811,504,853,637,867,333,775,212,762.9653,196.7913,696.0347,112.2087,815,236,864,273,821,204,729,220,704,332,675,269,3.235820,pushing car,transportation
17369,0,072772110.jpg,562,617,558,539,553,452,511,458,529,536,541,610,532,455,531,378,528.9869,352.8365,525.0131,303.1635,558,411,565,393,572,377,490,378,470,411,461,414,1.494952,pushing car,transportation
17370,0,039361034.jpg,496,377,466,322,510,278,540,280,558,331,596,368,525,279,454,252,451.3169,251.4431,403.6831,241.5569,378,311,404,288,441,253,466,251,436,293,410,325,1.459468,pushing car,transportation


This dataset contains a lot of categories that don't involve much movement, and have obscured limbs. I'm removing like "inactivity quiet/light" as I don't expect them to be as useful. *Note: variables and files are named something along the lines of sports as the original idea was to just use the sports category.

In [126]:
Sports = set()
for i in dataset.index:
  if dataset["Category"][i] == "fishing and hunting" or dataset["Category"][i] == "home activities" or dataset["Category"][i] == "home repair" or dataset["Category"][i] == "inactivity quiet/light" or dataset["Category"][i] == "lawn and garden" or dataset["Category"][i] == "miscellaneous" or dataset["Category"][i] == "music playing" or dataset["Category"][i] == "occupation" or dataset["Category"][i] == "religious activities" or dataset["Category"][i] == "self care" or dataset["Category"][i] == "transportation" or dataset["Category"][i] == "volunteer activities":
    dataset.drop(i, inplace=True)
  else:
    Sports.add(dataset["NAME"][i])

In [127]:
dataset

,Unnamed: 0,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,l ankle_X,l ankle_Y,pelvis_X,pelvis_Y,thorax_X,thorax_Y,upper neck_X,upper neck_Y,head top_X,head top_Y,r wrist_X,r wrist_Y,r elbow_X,r elbow_Y,r shoulder_X,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,0,015601864.jpg,620,394,616,269,573,185,647,188,661,221,656,231,610,187,647,176,637.0201,189.8183,695.9799,108.1817,606,217,553,161,601,167,692,185,693,240,688,313,3.021046,curling,sports
1,0,015599452.jpg,-1,-1,-1,-1,806,543,720,593,-1,-1,-1,-1,763,568,683,290,682.0000,256.0000,676.0000,68.0000,563,296,555,410,647,281,719,299,711,516,545,466,5.641276,curling,sports
2,0,005808361.jpg,804,711,816,510,908,438,1040,454,906,528,883,707,974,446,985,253,982.7591,235.9694,962.2409,80.0306,869,214,798,340,902,253,1067,253,1167,353,1142,478,4.718488,curling,sports
3,0,086617615.jpg,301,461,305,375,201,340,294,342,335,370,331,455,248,341,279,263,277.0210,268.7786,305.9790,184.2214,328,354,260,335,244,261,314,264,327,320,362,346,2.681349,curling,sports
4,0,060111501.jpg,980,322,896,318,865,248,943,226,948,290,881,349,904,237,858,135,871.1877,180.4244,835.8123,58.5756,772,294,754,247,792,147,923,123,995,163,961,223,3.806403,curling,sports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17311,0,012741552.jpg,855,305,860,233,850,167,871,173,878,245,853,299,861,170,879,115,883.5063,107.4895,904.4937,72.5105,870,170,837,146,860,105,898,125,894,166,900,195,1.223765,"moto-cross, off-road motor sports, all-terrain...",sports
17312,0,089258378.jpg,542,298,623,214,608,132,650,165,664,256,588,359,629,149,741,92,740.1674,92.2345,812.8326,71.7655,674,175,640,121,715,87,766,96,736,154,755,220,2.264791,"moto-cross, off-road motor sports, all-terrain...",sports
17313,0,076912890.jpg,761,679,756,618,756,551,807,561,801,634,796,711,782,556,767,406,765.6631,390.9975,759.3369,320.0025,610,323,645,368,711,391,822,420,855,501,860,567,2.138291,"moto-cross, off-road motor sports, all-terrain...",sports
17314,0,099616030.jpg,704,336,693,284,744,281,742,301,685,323,696,363,743,291,775,250,775.1853,238.3288,775.8147,198.6712,716,238,759,252,776,240,773,259,766,292,739,281,1.189877,"moto-cross, off-road motor sports, all-terrain...",sports


In [128]:
len(dataset)

9002

9002 should be enough for now.
Save this to csv for later processing.

In [129]:
dataset.to_csv("sports_annotations_no_box.csv")

In [131]:
!rm -rf images_data

In [132]:
!mkdir images_data

In [133]:
for image in os.listdir('images'):
  if image in Sports:
    shutil.copy(f'images/{image}', 'images_data')

In [134]:
len(os.listdir('images_data'))

9002

Lastly, I download and save this data to Google Drive for easier access in other files/programs. These files will also be found in the respository.

In [135]:
from google.colab import drive

In [136]:
drive.mount('/content/drive')
with open(f'/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/sports_annotations_no_box.csv', 'wb'):
  shutil.copy("sports_annotations_no_box.csv", "/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/sports_annotations_no_box.csv")
drive.flush_and_unmount()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [138]:
!tar cfz images.tar.gz images_data/
drive.mount('/content/drive')

with open(f'/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/images.tar.gz', 'wb'):
  shutil.copy('images.tar.gz', '/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/images.tar.gz')
# for fn in Sports:
#   with open(f'/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/Images/{fn}', 'wb'):
#     shutil.copy(f"images_data/{fn}", f"/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/Images/{fn}")

drive.flush_and_unmount()

Mounted at /content/drive
